# Module 3 Homework - Data Warehousing with DuckDB

This notebook contains solutions to Module 3 homework using DuckDB instead of BigQuery.

**Dataset**: Yellow Taxi Trip Records for January 2024 - June 2024

## Setup and Data Loading

In [ ]:
import duckdb
import pandas as pd
from pathlib import Path

# Initialize DuckDB connection
con = duckdb.connect('homework.duckdb')

print("DuckDB version:", duckdb.__version__)

### Create External Table (similar to BigQuery External Table)

In DuckDB, we can query Parquet files directly without loading them into memory - this is similar to BigQuery's External Tables.

In [ ]:
# Create a view that reads directly from parquet files (like an external table)
con.execute("""
CREATE OR REPLACE VIEW yellow_taxi_external AS
SELECT * FROM read_parquet('data/yellow_tripdata_2024-*.parquet')
""")

print("External view created successfully")

# Preview the data
con.execute("SELECT * FROM yellow_taxi_external LIMIT 5").df()

### Create Materialized Table (similar to BigQuery Table)

Now let's create a regular table by loading all data into DuckDB - this is similar to a materialized table in BigQuery.

In [ ]:
# Create a materialized table (data loaded into DuckDB)
con.execute("""
CREATE OR REPLACE TABLE yellow_taxi_materialized AS
SELECT * FROM read_parquet('data/yellow_tripdata_2024-*.parquet')
""")

print("Materialized table created successfully")

## Question 1: Total Record Count

**Question**: What is count of records for the 2024 Yellow Taxi Data?

Options:
- 65,623
- 840,402
- 20,332,093
- 85,431,289

In [ ]:
# Count total records
result = con.execute("""
SELECT COUNT(*) as total_records
FROM yellow_taxi_materialized
""").df()

total_records = result['total_records'][0]
print(f"Total records: {total_records:,}")

result

**Answer**: 20,332,093

## Question 2: Distinct PULocationIDs - External vs Materialized

**Question**: Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables. What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

**Note**: DuckDB doesn't provide byte estimates like BigQuery, but we can observe the conceptual difference:
- External view: Reads directly from Parquet files (only the column needed)
- Materialized table: Data already in DuckDB storage

In [ ]:
# Query on external view
result_external = con.execute("""
SELECT COUNT(DISTINCT PULocationID) as distinct_pu_locations
FROM yellow_taxi_external
""").df()

print("External view result:")
print(result_external)

# Query on materialized table
result_materialized = con.execute("""
SELECT COUNT(DISTINCT PULocationID) as distinct_pu_locations
FROM yellow_taxi_materialized
""").df()

print("\nMaterialized table result:")
print(result_materialized)

**Answer for BigQuery**: 0 MB for the External Table and 155.12 MB for the Materialized Table

**Explanation**: 
- External tables in BigQuery don't cache metadata, so the estimate is 0 MB
- Materialized tables have full metadata, showing the actual data size

## Question 3: Columnar Storage - One vs Two Columns

**Question**: Write a query to retrieve the PULocationID from the table. Now write a query to retrieve the PULocationID and DOLocationID on the same table. Why are the estimated number of Bytes different?

**Answer**: BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.

In [ ]:
# Query with one column
result_one = con.execute("""
SELECT PULocationID
FROM yellow_taxi_materialized
LIMIT 10
""").df()

print("One column query:")
print(result_one)

# Query with two columns
result_two = con.execute("""
SELECT PULocationID, DOLocationID
FROM yellow_taxi_materialized
LIMIT 10
""").df()

print("\nTwo column query:")
print(result_two)

**Explanation**: Both BigQuery and DuckDB use columnar storage. When you query one column, only that column's data is scanned. When you query two columns, both columns' data must be scanned, roughly doubling the bytes processed.

## Question 4: Zero Fare Amount

**Question**: How many records have a fare_amount of 0?

Options:
- 128,210
- 546,578
- 20,188,016
- 8,333

In [ ]:
# Count records with fare_amount = 0
result = con.execute("""
SELECT COUNT(*) as zero_fare_count
FROM yellow_taxi_materialized
WHERE fare_amount = 0
""").df()

zero_fare_count = result['zero_fare_count'][0]
print(f"Records with fare_amount = 0: {zero_fare_count:,}")

result

**Answer**: 8,333

## Question 5: Optimization Strategy - Partitioning and Clustering

**Question**: What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID?

**Answer**: Partition by tpep_dropoff_datetime and Cluster on VendorID

**Explanation**: 
- Partition by the date column used for filtering (tpep_dropoff_datetime) - this reduces the amount of data scanned
- Cluster by the column used for ordering (VendorID) - this optimizes sorting operations

In DuckDB, we don't have the same partitioning/clustering as BigQuery, but we can demonstrate the concept by creating an ordered table:

In [ ]:
# Create an optimized table with data ordered by dropoff datetime and VendorID
con.execute("""
CREATE OR REPLACE TABLE yellow_taxi_optimized AS
SELECT * 
FROM yellow_taxi_materialized
ORDER BY tpep_dropoff_datetime, VendorID
""")

print("Optimized table created (sorted by tpep_dropoff_datetime and VendorID)")

# Verify the table
con.execute("""
SELECT COUNT(*) as record_count,
       MIN(tpep_dropoff_datetime) as min_dropoff,
       MAX(tpep_dropoff_datetime) as max_dropoff
FROM yellow_taxi_optimized
""").df()

## Question 6: Query Performance - Non-Partitioned vs Partitioned

**Question**: Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime 2024-03-01 and 2024-03-15 (inclusive). Compare the estimated bytes between the materialized table and the partitioned table.

**Answer**: 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table

In [ ]:
# Query on regular materialized table
result_regular = con.execute("""
SELECT DISTINCT VendorID
FROM yellow_taxi_materialized
WHERE tpep_dropoff_datetime >= '2024-03-01'
  AND tpep_dropoff_datetime < '2024-03-16'
ORDER BY VendorID
""").df()

print("Non-partitioned table result:")
print(result_regular)

# Query on optimized (sorted) table
result_optimized = con.execute("""
SELECT DISTINCT VendorID
FROM yellow_taxi_optimized
WHERE tpep_dropoff_datetime >= '2024-03-01'
  AND tpep_dropoff_datetime < '2024-03-16'
ORDER BY VendorID
""").df()

print("\nOptimized (sorted) table result:")
print(result_optimized)

**Explanation**: In BigQuery, partitioning by date allows the query optimizer to skip entire partitions that don't match the date range. For a query filtering on March 1-15, a partitioned table only scans ~2 weeks of data, while a non-partitioned table must scan the entire 6-month dataset.

## Question 7: External Table Storage Location

**Question**: Where is the data stored in the External Table you created?

Options:
- Big Query
- Container Registry
- GCP Bucket
- Big Table

**Answer**: GCP Bucket

**Explanation**: External tables in BigQuery reference data stored in external sources like GCS (Google Cloud Storage) buckets. The data remains in the bucket and is not copied into BigQuery storage. Similarly, our DuckDB external view reads directly from the Parquet files on disk.

In [ ]:
# In our case, the external view reads from local Parquet files
from pathlib import Path

data_files = list(Path('data').glob('yellow_tripdata_2024-*.parquet'))
print("External data files:")
for f in data_files:
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"  {f.name}: {size_mb:.2f} MB")

total_size = sum(f.stat().st_size for f in data_files) / (1024 * 1024)
print(f"\nTotal external data size: {total_size:.2f} MB")

## Question 8: Clustering Best Practice

**Question**: It is best practice in Big Query to always cluster your data:

**Answer**: False

**Explanation**: Clustering is not always beneficial. It's most useful when:
- You frequently filter or aggregate by specific columns
- Your table is large (> 1 GB)
- Your queries filter on high-cardinality columns

For small tables or tables without common filter patterns, clustering adds overhead without significant benefit.

## Question 9 (Bonus): COUNT(*) on Materialized Table

**Question**: Write a SELECT count(*) query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

In [ ]:
# COUNT(*) query
result = con.execute("""
SELECT COUNT(*) as total_count
FROM yellow_taxi_materialized
""").df()

print("Count result:")
print(result)

**Answer**: In BigQuery, the estimate would be 0 MB.

**Explanation**: BigQuery maintains metadata about tables including the row count. A simple `COUNT(*)` query without any WHERE clause or column references can be answered directly from metadata without scanning any actual data. This is why the estimated bytes is 0 MB - the query doesn't need to read the table data at all.

## Summary of Answers

1. **Question 1**: 20,332,093 records
2. **Question 2**: 0 MB for the External Table and 155.12 MB for the Materialized Table
3. **Question 3**: BigQuery is a columnar database - querying two columns requires reading more data than one column
4. **Question 4**: 8,333 records with fare_amount = 0
5. **Question 5**: Partition by tpep_dropoff_datetime and Cluster on VendorID
6. **Question 6**: 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table
7. **Question 7**: GCP Bucket
8. **Question 8**: False
9. **Question 9 (Bonus)**: 0 MB - BigQuery uses metadata for simple COUNT(*) queries

## Additional Analysis with DuckDB

In [ ]:
# Get basic statistics about the dataset
stats = con.execute("""
SELECT 
    COUNT(*) as total_trips,
    COUNT(DISTINCT VendorID) as unique_vendors,
    COUNT(DISTINCT PULocationID) as unique_pickup_locations,
    COUNT(DISTINCT DOLocationID) as unique_dropoff_locations,
    MIN(tpep_pickup_datetime) as earliest_pickup,
    MAX(tpep_pickup_datetime) as latest_pickup,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(AVG(trip_distance), 2) as avg_distance
FROM yellow_taxi_materialized
""").df()

print("Dataset Statistics:")
stats

In [ ]:
# Monthly trip distribution
monthly = con.execute("""
SELECT 
    EXTRACT(YEAR FROM tpep_pickup_datetime) as year,
    EXTRACT(MONTH FROM tpep_pickup_datetime) as month,
    COUNT(*) as trip_count,
    ROUND(AVG(fare_amount), 2) as avg_fare
FROM yellow_taxi_materialized
GROUP BY year, month
ORDER BY year, month
""").df()

print("Monthly Trip Distribution:")
monthly

In [ ]:
# Close the connection
con.close()
print("\nDuckDB connection closed")